In [1]:
from __future__ import print_function # so print doesn't show brackets
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import time as time
import random
import pickle
pickle.HIGHEST_PROTOCOL = 2
sys.path.append(os.path.join("..", "Libraries","QML_lib"))
from RedisSettings import *
import Evo as evo
import DataBase 
from QMD import QMD #  class moved to QMD in Library
import QML
import ModelGeneration 
import BayesF
import matplotlib.pyplot as plt
%matplotlib inline
from pympler import asizeof
import matplotlib.pyplot as plt
paulis = ['x', 'y', 'z'] # will be chosen at random. or uncomment below and comment within loop to hard-set

import time as time 
import argparse
parser = argparse.ArgumentParser(description='Pass variables for (I)QLE.')

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [ ]:
num_runs=1
num_particles=1000
num_experiments = 400
num_times_bayes = 'all'
qle=True

# initial_op_list = ['x', 'y', 'z', 'xTx', 'yTz', 'xTz']
initial_op_list = ['y', 'xTyPPzTx', 'xPy', 'zPy', 'xTy']
initial_op_list = ['xPy', 'y']
#true_op = random.choice(initial_op_list)
#true_params = [np.random.rand() for i in range(n_pars)]

num_ops = len(initial_op_list)

for i in range(num_ops):
#    true_op=initial_op_list[i%2]
    true_op= initial_op_list[0]
    #true_op = 'xPzPy'
    true_params = [np.random.rand()]
    qmd = QMD(
        initial_op_list=initial_op_list, 
        true_operator=true_op, 
        true_param_list=None, 
        num_particles=num_particles,
        num_experiments = num_experiments, 
        num_times_for_bayes_updates = num_times_bayes,
        qle=qle,
        num_probes=5,
        max_num_branches = 0,
        max_num_qubits = 2, 
        parallel = True
    )
   # qmd.learnModelNameList(model_name_list=['z'], blocking=True, use_rq=False)
    qmd.blockingQMD()

In [3]:
qmd_info_db.keys()

['rq:worker:IT067176.8034']

In [2]:
qmd_info = pickle.loads(qmd_info_db.get('QMDInfo'))


TypeError: StringIO() argument 1 must be string or buffer, not None

In [ ]:
qmd_info

In [ ]:
print("%.0e"%4254352435423)